# Analysis Sparsity

Learning to find the optimal dictionary $D$ s.t. $\min_x ||Dx||_\epsilon + \frac{1}{2}||x-f||^2$ is a good denoising of data $f$.

## Imports:

In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torchvision

import matplotlib.pyplot as plt
import plotly.offline as ply
import plotly.graph_objs as go
ply.offline.init_notebook_mode(connected=True)

import os
import time
import datetime
from collections import defaultdict
from copy import deepcopy

from IPython.display import clear_output
%load_ext autoreload

from PIL import Image
import bilevelsurrogates as Sur

#ymmv:
torch.backends.cudnn.benchmark = True

### GPU Setup:

In [2]:
device = torch.device('cuda:1')
dtype = torch.float

In [3]:
# Sur.deterministic() # Disable this for reasonable timings, enable to reproduce numbers from paper
# Note that the actual timing depends on the number of iterations of the subproblems, which can vary.

## Data Generation:

Download the data first!!

In [4]:
noise = 25/255
clip = False
batch_size = 200

patch_cropping = torchvision.transforms.RandomCrop((64,64))
#patch_cropping = None

path = os.getcwd() + '/data/BSDS300/' 
datasetTrain = Sur.data.BSDS300_for_denoising(path, noise_std=noise, augmentations=patch_cropping, flip=True,
                                              grayscale=True, clip_to_realistic=clip)
datasetTest = Sur.data.BSDS300_for_denoising(path, split = 'trainingZhang', noise_std=noise,
                                             augmentations=patch_cropping,
                                             flip=True, grayscale=True, clip_to_realistic=clip)
datasetTestFull = Sur.data.BSDS300_for_denoising(path, split = 'testing68Zhang', noise_std=noise,
                                             augmentations=None, flip=True, grayscale=True,
                                             clip_to_realistic=clip)
datasetTest12 = Sur.data.Test12_for_denoising(os.path.dirname(os.getcwd()) + '/data/test12/', noise_std=noise,
                                             augmentations=None, clip_to_realistic=clip)
samples = Sur.data.Samples(datasetTrain, batch_size, device=device, dtype=dtype)

FileNotFoundError: [Errno 2] No such file or directory: '/home/vsa_jonas/Dropbox/Documents_Hyperion/Python/ParametricMajorizationICCVVersion/data/BSDS300/iids_train.txt'

### Example training patches

In [ ]:
fig, axes = plt.subplots(2,4,figsize=(20, 10))
for idx, img_id in enumerate(np.random.choice(batch_size,4)):
    axes[0,idx].imshow(datasetTrain.unnormalize(samples.x[img_id,0,:,:]).cpu(), cmap='gray')
    axes[1,idx].imshow(datasetTrain.unnormalize(samples.y[img_id,0,:,:]).cpu(), cmap='gray')
fig.canvas.draw()
print(f'noisy psnr: {Sur.psnr_compute(samples.y, samples.x)}')

### Example test images

In [ ]:
fig, axes = plt.subplots(2,4,figsize=(20, 10))
for idx, img_id in enumerate(np.random.choice(len(datasetTest),4)):
    test_gt, test_noisy = datasetTest[img_id]
    axes[0,idx].imshow(datasetTrain.unnormalize(test_gt[0,:,:]).cpu(), cmap='gray')
    axes[1,idx].imshow(datasetTrain.unnormalize(test_noisy[0,:,:]).cpu(), cmap='gray')
fig.canvas.draw()

# Define Energy Model

In [ ]:
model_setup = dict()
model_setup['data'] = dict(dtype=dtype, device=device)

# Sizes:
model_setup['x_dims'] = samples.x.shape

# Hyperparameters:
model_setup['alpha'] = 0.01
model_setup['epsilon'] = 0.0
model_setup['bias'] = False
model_setup['norm'] = 'aniso'
model_setup['clip'] = True

model_setup['inference'] = dict(tolerance=1e-4, max_iterations=250)


# Setups 1, 2, 3 correspond to table 1 in the paper:
setup = 2

# Define Model
def energy_setup(model_setup):
    if setup == 1:
        dictionary = Sur.DCTConvolution(in_channels=1, out_channels=3, kernel_size=3, mean=False,
                                        bias=model_setup['bias']).to(**model_setup['data'])
    elif setup == 2:
        dictionary = Sur.DCTConvolution(in_channels=1, out_channels=48, kernel_size=7, mean=False,
                                        bias=model_setup['bias']).to(**model_setup['data'])
    elif setup == 3:
        dictionary = Sur.DCTConvolution(in_channels=1, out_channels=96, kernel_size=9, mean=False,
                                        bias=model_setup['bias']).to(**model_setup['data'])
        model_setup['alpha'] *= (48*7*7) / (96*9*9)
    elif setup == 4:
        dictionary = Sur.Gradient(in_channels=1, scalable=True).to(**model_setup['data'])
    elif setup == 5:
        dictionary = Sur.DCTConvolution(in_channels=1, out_channels=168, kernel_size=13, mean=False,
                                        bias=model_setup['bias']).to(**model_setup['data'])
        model_setup['alpha'] *= (48*7*7) / (168*13**2)    
    energy = Sur.model.AnalysisSparsity(dictionary, setup=model_setup)
    return energy
energy = energy_setup(model_setup)

## Define Higher-level loss

In [ ]:
loss = Sur.loss.PSNR()

# --- Training Algorithms ---

### Visualize initial dictionary

In [ ]:
Sur.visualize(energy.operator);

In [ ]:
# torch.backends.cudnn.deterministic = False # enable this for non-deterministic timings with det. data selection and init
# torch.backends.cudnn.benchmark = True # enable this for non-deterministic timings with det. data selection and init

# 1) Single step Parametric Majorizer

### Parameters:

In [ ]:
algorithm = 'joint-dual'
training_setup = Sur.training.default_setup('DiscriminativeLearning', algorithm)

### Training:

In [ ]:
energy = energy_setup(model_setup)
optimizer = Sur.training.DiscriminativeLearning(energy, loss, samples, training_setup, algorithm=algorithm)
optimizer.run();

### Output

In [ ]:
Sur.visualize(energy.operator)
img_denoised, _ = Sur.training_error(energy, loss, samples);
samples.redraw_noise()
img_denoised, _ = Sur.training_error(energy, loss, samples);

In [ ]:
_, _ = Sur.testing_error(energy, loss, datasetTest, batch_size=25);
_, _ = Sur.testing_error(energy, loss, datasetTestFull);
_, _ = Sur.testing_error(energy, loss, datasetTest12, batch_size=1);

### Visualization

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(20, 10))
axes[0].imshow(samples.x[0,:,:,].permute(1, 2, 0).cpu().numpy().squeeze(), cmap='gray')
axes[1].imshow(samples.y[0,:,:,].permute(1, 2, 0).cpu().numpy().squeeze(), cmap='gray')
axes[2].imshow(img_denoised[0,:,:,:].permute(1, 2, 0).cpu().numpy().squeeze(), cmap='gray')
fig.canvas.draw()

In [ ]:
layout = go.Layout(
         xaxis=dict(
             type='log',
             autorange=True,
             title='Iterations',
             automargin=True
             ),
         yaxis=dict(
             type='log',
             autorange  = True,
             title='Value',
             automargin=True
         ),
         )

data = [go.Scatter(y=optimizer.stats['loss'], name = 'Discriminator loss')]
fig = go.Figure(data=data, layout=layout)
ply.iplot(fig)

# 2) Iterated Parametric Majorization

In [ ]:
iterative_setup = Sur.training.default_setup('IterativeLearning', algorithm)
iterative_setup

In [ ]:
# Define Model
energy = energy_setup(model_setup)
subroutine = Sur.training.DiscriminativeLearning(energy, loss, samples, training_setup, algorithm=algorithm)
optimizer =  Sur.training.IterativeLearning(subroutine, iterative_setup)
optimizer.run();

In [ ]:
Sur.visualize(energy.operator);

### Validate training loss

In [ ]:
img_denoised, _ = Sur.training_error(energy, loss, samples);
samples.redraw_noise()
img_denoised, _ = Sur.training_error(energy, loss, samples);

In [ ]:
samples.reset()
samples.step()
img_denoised, _ = Sur.training_error(energy, loss, samples);

### Save Model

In [ ]:
data = model_setup, training_setup, iterative_setup, energy.state_dict()
path = './energies/'+algorithm+str(datetime.date.today()) + '.pt'
#torch.save(data, path)

### Test 

In [ ]:
_, _ = Sur.testing_error(energy, loss, datasetTest, batch_size=25);
_, _ = Sur.testing_error(energy, loss, datasetTestFull, batch_size=4);
_, _ = Sur.testing_error(energy, loss, datasetTest12, batch_size=1);

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(20, 10))
axes[0].imshow(samples.x[0,:,:,].permute(1, 2, 0).cpu().numpy().squeeze(), cmap='gray')
axes[1].imshow(samples.y[0,:,:,].permute(1, 2, 0).cpu().numpy().squeeze(), cmap='gray')
axes[2].imshow(img_denoised[0,:,:,:].permute(1, 2, 0).cpu().numpy().squeeze(), cmap='gray')
fig.canvas.draw()